# Class Demo — Apriori (Frequent Itemsets Only): Candidate Generation + Pruning

This notebook is **Apriori-focused** (frequent itemsets only), emphasizing:
- **level-wise search** (k = 1 → 2 → 3 → …),
- **candidate generation** (Ck) and **frequent itemsets** (Lk),
- **Apriori property (downward closure)** for **pruning**.

> **No FP-Growth** and **no association rules** in this demo.

---

## Learning goals
By the end of this demo, you will be able to:
1. Compute supports for 1-itemsets and identify **L1**.
2. Generate candidates **C2**, compute supports, and obtain **L2**.
3. Generate candidates **C3** and prune them using the Apriori property.
4. Verify results using `mlxtend` Apriori.


## 0) Setup


In [1]:
import pandas as pd
import numpy as np
from itertools import combinations

# If needed (e.g., Colab), uncomment:
# !pip -q install mlxtend

from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori


## 1) Transaction Dataset (Market Basket)

We use a small dataset so we can verify supports by inspection.


In [2]:
transactions = [
    ["milk", "bread", "eggs"],
    ["milk", "bread"],
    ["milk", "diapers", "beer", "bread"],
    ["bread", "butter"],
    ["milk", "bread", "butter"],
    ["diapers", "beer"],
    ["milk", "diapers", "bread", "butter"],
    ["bread", "eggs"],
    ["milk", "eggs"],
    ["milk", "bread", "diapers", "beer"],
]

n = len(transactions)
print("Number of transactions:", n)
print("T1:", transactions[0])


Number of transactions: 10
T1: ['milk', 'bread', 'eggs']


### 1.1) View transactions (optional)


In [3]:
tx_df = pd.DataFrame({
    "TID": [f"T{i+1:02d}" for i in range(n)],
    "items": [", ".join(t) for t in transactions]
})
tx_df


,TID,items
0,T01,"milk, bread, eggs"
1,T02,"milk, bread"
2,T03,"milk, diapers, beer, bread"
3,T04,"bread, butter"
4,T05,"milk, bread, butter"
5,T06,"diapers, beer"
6,T07,"milk, diapers, bread, butter"
7,T08,"bread, eggs"
8,T09,"milk, eggs"
9,T10,"milk, bread, diapers, beer"


## 2) Minimum support (minsup)

\[
support(X) = \frac{\#(transactions\ containing\ X)}{N}
\]

With N=10:
- minsup = 0.30 → min_count = 3 transactions


In [4]:
minsup = 0.30
min_count = int(np.ceil(minsup * n))
print(f"minsup = {minsup:.2f}  ->  min_count = {min_count} transactions")


minsup = 0.30  ->  min_count = 3 transactions


## 3) Support counting helpers


In [5]:
def support_count(itemset, transactions):
    itemset = set(itemset)
    return sum(1 for t in transactions if itemset.issubset(set(t)))

def support(itemset, transactions):
    return support_count(itemset, transactions) / len(transactions)

def pretty(itemset):
    return "{" + ", ".join(sorted(itemset)) + "}"


## 4) Level k = 1: Build C1 and L1

- **C1**: all single-item candidates  
- **L1**: candidates in C1 whose support ≥ minsup


In [6]:
items = sorted({it for t in transactions for it in t})
C1 = [frozenset([i]) for i in items]

rows = []
for x in C1:
    cnt = support_count(x, transactions)
    rows.append({
        "C1 candidate": pretty(x),
        "count": cnt,
        "support": cnt / n,
        "in L1?": cnt >= min_count
    })

C1_table = pd.DataFrame(rows).sort_values(["count","C1 candidate"], ascending=[False, True]).reset_index(drop=True)
C1_table


,C1 candidate,count,support,in L1?
0,{bread},8,0.8,True
1,{milk},7,0.7,True
2,{diapers},4,0.4,True
3,{beer},3,0.3,True
4,{butter},3,0.3,True
5,{eggs},3,0.3,True


In [7]:
L1 = [set(row["C1 candidate"].strip("{}").split(", ")) for _, row in C1_table.iterrows() if row["in L1?"]]
L1 = [set([i for i in s if i]) for s in L1]

print("L1:")
for s in L1:
    print(" ", pretty(s))


L1:
  {bread}
  {milk}
  {diapers}
  {beer}
  {butter}
  {eggs}


## 5) Level k = 2: Generate C2 from L1, then compute L2

- **Join step:** combine pairs of frequent 1-itemsets → C2  
- **Count support** for each candidate  
- Keep those meeting minsup → **L2**


In [8]:
C2 = []
for a, b in combinations(L1, 2):
    u = a | b
    if len(u) == 2:
        C2.append(frozenset(u))

C2 = sorted(set(C2), key=lambda s: sorted(list(s)))

rows = []
for x in C2:
    cnt = support_count(x, transactions)
    rows.append({
        "C2 candidate": pretty(x),
        "count": cnt,
        "support": cnt / n,
        "in L2?": cnt >= min_count
    })

C2_table = pd.DataFrame(rows).sort_values(["count","C2 candidate"], ascending=[False, True]).reset_index(drop=True)
C2_table


,C2 candidate,count,support,in L2?
0,"{bread, milk}",6,0.6,True
1,"{beer, diapers}",3,0.3,True
2,"{bread, butter}",3,0.3,True
3,"{bread, diapers}",3,0.3,True
4,"{diapers, milk}",3,0.3,True
5,"{beer, bread}",2,0.2,False
6,"{beer, milk}",2,0.2,False
7,"{bread, eggs}",2,0.2,False
8,"{butter, milk}",2,0.2,False
9,"{eggs, milk}",2,0.2,False


In [9]:
L2 = [set(row["C2 candidate"].strip("{}").split(", ")) for _, row in C2_table.iterrows() if row["in L2?"]]
L2 = [set([i for i in s if i]) for s in L2]

print("L2:")
for s in L2:
    print(" ", pretty(s))


L2:
  {bread, milk}
  {beer, diapers}
  {bread, butter}
  {bread, diapers}
  {diapers, milk}


## 6) Level k = 3: Generate C3 with Apriori pruning, then compute L3

### Join step (L2 ⨝ L2)
Form candidates by taking unions of frequent 2-itemsets that produce 3 items.

### Prune step (Apriori property)
A 3-item candidate is kept only if **all its 2-item subsets** are in L2.


In [10]:
L2_frozen = set(frozenset(s) for s in L2)

# Join: unions that create size-3
C3_raw = set()
for a, b in combinations(L2, 2):
    u = a | b
    if len(u) == 3:
        C3_raw.add(frozenset(u))

# Prune: all 2-subsets must be frequent (in L2)
C3 = []
for c in C3_raw:
    ok = True
    for sub in combinations(list(c), 2):
        if frozenset(sub) not in L2_frozen:
            ok = False
            break
    if ok:
        C3.append(c)

C3 = sorted(set(C3), key=lambda s: sorted(list(s)))

print("C3 candidates after Apriori pruning:")
for s in C3:
    print(" ", pretty(s))


C3 candidates after Apriori pruning:
  {bread, diapers, milk}


In [11]:
rows = []
for x in C3:
    cnt = support_count(x, transactions)
    rows.append({
        "C3 candidate": pretty(x),
        "count": cnt,
        "support": cnt / n,
        "in L3?": cnt >= min_count
    })

C3_table = pd.DataFrame(rows).sort_values(["count","C3 candidate"], ascending=[False, True]).reset_index(drop=True)
C3_table


,C3 candidate,count,support,in L3?
0,"{bread, diapers, milk}",3,0.3,True


In [12]:
L3 = [set(row["C3 candidate"].strip("{}").split(", ")) for _, row in C3_table.iterrows() if row["in L3?"]]
L3 = [set([i for i in s if i]) for s in L3]

print("L3:")
if not L3:
    print("  (none at this minsup)")
else:
    for s in L3:
        print(" ", pretty(s))


L3:
  {bread, diapers, milk}


## 7) Summary of frequent itemsets (manual Apriori)

We summarize L1, L2, and L3.


In [13]:
def summarize(Lk, k):
    return pd.DataFrame({"k": [k]*len(Lk), "itemset": [pretty(s) for s in Lk]})

summary_manual = pd.concat([summarize(L1,1), summarize(L2,2), summarize(L3,3)], ignore_index=True)
summary_manual


,k,itemset
0,1,{bread}
1,1,{milk}
2,1,{diapers}
3,1,{beer}
4,1,{butter}
5,1,{eggs}
6,2,"{bread, milk}"
7,2,"{beer, diapers}"
8,2,"{bread, butter}"
9,2,"{bread, diapers}"


## 8) Sanity check with `mlxtend` Apriori

This confirms the frequent itemsets for the same minsup (ordering may differ).


In [14]:
te = TransactionEncoder()
basket = pd.DataFrame(te.fit(transactions).transform(transactions), columns=te.columns_)

freq_ml = apriori(basket, min_support=minsup, use_colnames=True)
freq_ml = freq_ml.sort_values(["support","itemsets"], ascending=[False, True]).reset_index(drop=True)

freq_ml_display = freq_ml.copy()
freq_ml_display["itemsets"] = freq_ml_display["itemsets"].apply(lambda s: "{" + ", ".join(sorted(list(s))) + "}")
freq_ml_display


,support,itemsets
0,0.8,{bread}
1,0.7,{milk}
2,0.6,"{bread, milk}"
3,0.4,{diapers}
4,0.3,{beer}
5,0.3,{butter}
6,0.3,{eggs}
7,0.3,"{beer, diapers}"
8,0.3,"{bread, butter}"
9,0.3,"{bread, diapers}"


## 9) In-class prompts (Apriori-specific)

1. Where exactly did pruning happen at k=3? Which candidates were removed and why?  
2. Why does Apriori become expensive when minsup is small?  
3. Which part is costly: generating candidates, counting support, or both?
